In [32]:
import os
from pydub import AudioSegment
import re
import numpy as np
import pandas as pd
from datetime import timedelta

In [33]:
d_path = "diarized_audio/raw_diarizations"
diarization_paths = {}
for directory in os.listdir(d_path):
    diarization_paths[directory] = os.path.join(d_path, directory)

`Careful!` -- the following cell deletes previous work. If intended, run to make way for the new one

In [34]:
# # Remove all csv files from diarization directory
for path in os.listdir(d_path):
    !rm -r {os.path.join(d_path, path)}/speaker_turns.csv

rm: cannot remove 'diarized_audio/raw_diarizations/lexFridman-mrBeast-11012023/speaker_turns.csv': No such file or directory
rm: cannot remove 'diarized_audio/raw_diarizations/flagrant-joeRogan-07062022/speaker_turns.csv': No such file or directory
rm: cannot remove 'diarized_audio/raw_diarizations/JRE-wizKhalifa-31032019/speaker_turns.csv': No such file or directory


Utils:

In [35]:
def sort_directories(arr):
    def sort_key(s):
        return int(s.split('_')[1])

    return sorted(arr, key=sort_key)

def parse_time(time_str):
    # Parses the time string to a pandas.Timedelta object
    return pd.Timedelta(time_str)

def process_line(line):
    # Splits the line and extracts the required information
    time_data, speaker = line.split(']')
    time_data = time_data[2:]
    start_time, end_time = time_data.split(' -->  ')
    speaker = speaker.split(' ')[-1]  # Gets only the SPEAKER_xx part
    return speaker, parse_time(start_time), parse_time(end_time)

def most_common_letters(string, dictionary):
    def common_letter_count(s1, s2):
        return sum(min(s1.count(c), s2.count(c)) for c in set(s1))

    max_key = max(dictionary.keys(), key=lambda k: common_letter_count(k.lower(), string.lower()))
    return dictionary[max_key]

In [36]:
def get_speaker_turns(podcast_dictionaries, verbose : bool = False):
    diarizations = "diarized_audio/raw_diarizations"
    directories = [
        directory 
        for directory in os.listdir(diarizations) 
        if directory in podcast_dictionaries.keys()
    ]
    
    for podcast_name in directories:

        directory = os.path.join(diarizations, podcast_name)
        podcast_df = pd.DataFrame(columns=['speaker', 'start', 'end'])
        spacer_time = pd.Timedelta('0 days 00:00:00.998000')
        global_time = pd.Timedelta('0')
        splits = sort_directories([d for d in os.listdir(directory) if d.endswith('.txt')])
        #to_replace_dict = most_common_letters(podcast_name, podcast_dictionaries)
        to_replace_dict = podcast_dictionaries[podcast_name]
        
        if verbose: 
            print(directory)
            print(to_replace_dict)
        
        for idx, split in enumerate(splits):
            
            file_path = os.path.join(directory, split)
            # get split number with regex that searches for a number
            number = re.search(r'\d+(\.\d+)?', split).group()
            # get the speakers (SPEAKER_00, SPEAKER_...) that talk in this split

            # In case we want to discard the data in one of the splits, just go to the next split.
            if to_replace_dict[number] == "":
                continue

            real_speakers = to_replace_dict[number].split(";")

            with open(file_path, 'r') as file:
                for line in file:
                    speaker, start, end = process_line(line)

                    # Set the correct speaker

                    speaker = speaker.strip('\n')
                    for rsp in real_speakers:
                        rsp = rsp.split(",")
                        pyannote_speaker = 'SPEAKER_' + rsp[0]
                        
                        if pyannote_speaker == speaker:
                            speaker = rsp[1]
                    
                    # Adjust start and end to the global time
                    start = start + global_time - spacer_time
                    end = end + global_time - spacer_time

                    #Create a new tagged speaker intervention in the podcast
                    podcast_df.loc[len(podcast_df)] = [speaker, start, end]
            
            global_time = podcast_df["end"].max()
    
        pattern = 'SPEAKER_\d+'
        filtered_df = podcast_df[~podcast_df['speaker'].str.contains(pattern, na=False)]
        filtered_df.to_csv(os.path.join(directory, "speaker_turns.csv"), header=True, index=False)

In [37]:
%reload_ext autoreload
%autoreload 2
from speakers import replacer_dict

get_speaker_turns(replacer_dict, True)

diarized_audio/raw_diarizations/lexFridman-guidoVanRossum-26112022
{'1': '00,GUIDOVANROSSUM;01,LEXFRIDMAN', '2': '00,LEXFRIDMAN;01,GUIDOVANROSSUM', '3': '00,LEXFRIDMAN;01,GUIDOVANROSSUM;02,LEXFRIDMAN', '4': '00,LEXFRIDMAN;01,GUIDOVANROSSUM', '5': '00,LEXFRIDMAN;01,GUIDOVANROSSUM', '6': '00,LEXFRIDMAN;01,GUIDOVANROSSUM', '7': '00,LEXFRIDMAN;01,GUIDOVANROSSUM', '8': '00,GUIDOVANROSSUM;01,LEXFRIDMAN', '9': '00,GUIDOVANROSSUM;01,LEXFRIDMAN', '10': '00,LEXFRIDMAN;01,GUIDOVANROSSUM', '11': '00,GUIDOVANROSSUM;01,LEXFRIDMAN', '12': '00,LEXFRIDMAN;01,GUIDOVANROSSUM', '13': '00,GUIDOVANROSSUM;01,LEXFRIDMAN', '14': '00,GUIDOVANROSSUM;01,LEXFRIDMAN', '15': '00,LEXFRIDMAN;01,GUIDOVANROSSUM', '16': '00,LEXFRIDMAN;01,GUIDOVANROSSUM', '17': '00,GUIDOVANROSSUM;01,LEXFRIDMAN', '18': '00,LEXFRIDMAN;01,GUIDOVANROSSUM', '19': '00,LEXFRIDMAN;01,GUIDOVANROSSUM'}


diarized_audio/raw_diarizations/lexFridman-markZuckerberg-09062023
{'1': '00,LEXFRIDMAN;01,MARKZUCKERBERG', '2': '00,MARKZUCKERBERG;01,LEXFRIDMAN', '3': '00,LEXFRIDMAN;01,MARKZUCKERBERG', '4': '00,MARKZUCKERBERG;01,LEXFRIDMAN', '5': '00,MARKZUCKERBERG;01,LEXFRIDMAN', '6': '00,LEXFRIDMAN;01,MARKZUCKERBERG', '7': '00,LEXFRIDMAN;01,MARKZUCKERBERG', '8': '00,LEXFRIDMAN;01,MARKZUCKERBERG', '9': '00,LEXFRIDMAN;01,MARKZUCKERBERG', '10': '00,LEXFRIDMAN;01,MARKZUCKERBERG', '11': '00,MARKZUCKERBERG;01,LEXFRIDMAN', '12': '00,MARKZUCKERBERG;01,LEXFRIDMAN', '13': '00,MARKZUCKERBERG;01,LEXFRIDMAN', '14': '00,LEXFRIDMAN;01,MARKZUCKERBERG', '15': '00,LEXFRIDMAN;01,MARKZUCKERBERG', '16': '00,LEXFRIDMAN;01,MARKZUCKERBERG'}
diarized_audio/raw_diarizations/lexFridman-matthewMcConaughey-13062023
{'1': '00,MATTHEWMCCOUNAGHEY;01,LEXFRIDMAN', '2': '00,MATTHEWMCCOUNAGHEY;01,LEXFRIDMAN', '3': '00,MATTHEWMCCOUNAGHEY;01,LEXFRIDMAN', '4': '00,LEXFRIDMAN;01,MATTHEWMCCOUNAGHEY', '5': '00,MATTHEWMCCOUNAGHEY;01,LEXFRI

### Now we get what a speaker has said in text, linking the diarizations with the transcriptions

In [38]:
import pandas as pd

# Function to find the closest time in df2 to a given time in df1
def find_closest_start_time(df, given_time):
    # Calculate absolute time differences
    
    time_diff = (df['start'] - given_time).abs()
    
    # Find the index of the minimum difference
    closest_index = time_diff.idxmin()
    return closest_index

def find_closest_end_time(df, given_time):
    # Calculate absolute time differences
    
    time_diff = (df['end'] - given_time).abs()
    
    # Find the index of the minimum difference
    closest_index = time_diff.idxmin()
    return closest_index

def add_text_to_diarization(diarization_df, transcriptions_df):

    # New column for the combined text
    diarization_df['text'] = ''

    for index, row in diarization_df.iterrows():
        # Find closest start and end times in df2
        
        closest_start_index = find_closest_start_time(transcriptions_df, row['start'])
        closest_end_index = find_closest_end_time(transcriptions_df, row['end'])

        # Extract all rows in-between these indices
        if closest_start_index <= closest_end_index:
            relevant_text = transcriptions_df.loc[closest_start_index:closest_end_index, 'text']
        else:
            relevant_text = transcriptions_df.loc[closest_end_index:closest_start_index, 'text']

        # Combine the text and add to dataframe
        combined_text = "".join(relevant_text)
        diarization_df.at[index, 'text'] = combined_text

    # dataframe now contains the combined text in the new 'text' column
    return diarization_df


In [39]:
from nltk.tokenize import TweetTokenizer
from nltk import sent_tokenize

tweet_tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True)

#### Get all podcasts

Get speaker-tagged transcriptions for each dataframe, and save them in the datasets directory

In [40]:
diarizations = "diarized_audio/raw_diarizations"

directories = [
    directory
    for directory in os.listdir(diarizations) 
    if "speaker_turns.csv" in os.listdir(os.path.join(diarizations, directory))
]

for podcast_name in directories:
    directory = os.path.join(diarizations, podcast_name)

    diarization = pd.read_csv(f"diarized_audio/raw_diarizations/{podcast_name}/speaker_turns.csv")
    transcription = pd.read_csv(f"transcribed_audio/{podcast_name}_transcribed.csv")

    # Do a bit of data cleaning
    transcription['start'] = pd.to_timedelta(transcription['start'], unit='s')
    transcription['end'] = pd.to_timedelta(transcription['end'], unit='s')

    diarization['start'] = pd.to_timedelta(diarization['start'])
    diarization['end'] = pd.to_timedelta(diarization['end'])

    all_transcriptions = "".join(transcription["text"])

    final_df = add_text_to_diarization(diarization, transcription)
    final_df.to_csv(os.path.join("datasets", podcast_name + ".csv"), header=True, index=False)

    # speaked = final_df.groupby("speaker")['text'].apply(lambda x: ''.join(x))
    # print("Number of words:", len(tweet_tokenizer.tokenize(all_transcriptions)))
    # print("Number of sentences:", len(sent_tokenize(all_transcriptions)))
    # print(speaked)

Then join them all in a single dataframe.

In [41]:
df_name = "diarized_transcribed_df"
podcast_arr = []

for podcast_name in directories:
    podcast = pd.read_csv(os.path.join('datasets', podcast_name + '.csv'))
    podcast_arr.append(podcast)

diarized_transcribed_df = pd.concat(podcast_arr)
diarized_transcribed_df.to_csv(os.path.join("datasets", df_name + '.csv'), header=True, index=False)

In [42]:
text_df = diarized_transcribed_df.reset_index(drop=True)

### Preprocess the podcasts to ensure everything is correct

In [43]:
text_df['start'] = pd.to_timedelta(text_df['start'])
text_df['end'] = pd.to_timedelta(text_df['end'])

In [44]:
fix_df = (
    text_df[
        ~(( text_df['end'] - text_df['start'] ) < pd.Timedelta(1, unit='s'))
    ]
)

In [45]:
# Are there duplicates?
print(f"There are {fix_df['text'].duplicated().sum()} duplicates")

There are 228 duplicates


In [46]:
# Remove duplicates
duplicates_across_speakers = fix_df[fix_df.duplicated(subset='text', keep=False)]

unique_speakers_per_text = duplicates_across_speakers.drop_duplicates(subset=['text', 'speaker'])

texts_to_remove = unique_speakers_per_text[unique_speakers_per_text.duplicated(subset='text', keep=False)]['text']

# Remove entirely duplicate text if different speakers have said it
fix_df = fix_df[~fix_df['text'].isin(texts_to_remove)]

# Keep the first ocurrence of duplicate text if the same speaker has said it
# (it could be a catch phrase, or something that identifies the speaker, which is very useful for speaker prediction)
fix_df = fix_df.drop_duplicates(subset=['speaker', 'text'])

In [47]:
# Check that we have removed all the duplicates
fix_df['text'].duplicated().any()

False

In [48]:
fix_grouped_df = (
    fix_df.groupby('speaker')['text']
    .agg(
        lambda x: ''.join(x)
    ).to_frame().reset_index()
)

In [49]:
grouped_df = text_df.groupby('speaker')['text'].agg(lambda x: ''.join(x)).to_frame().reset_index()

In [50]:
grouped_df['words'] = grouped_df['text'].apply(lambda x: len(tweet_tokenizer.tokenize(x)))
grouped_df['sentences']  = grouped_df['text'].apply(lambda x: len(sent_tokenize(x)))

fix_grouped_df['words'] = fix_grouped_df['text'].apply(lambda x: len(tweet_tokenizer.tokenize(x)))
fix_grouped_df['sentences']  = fix_grouped_df['text'].apply(lambda x: len(sent_tokenize(x)))

In [51]:
wordiff = sum(grouped_df['words'] - fix_grouped_df['words'])
sentdiff = sum(grouped_df['sentences'] - fix_grouped_df['sentences'])

print(f"Eliminated Words = {wordiff} out of {grouped_df['words'].sum()} ({((wordiff / grouped_df['words'].sum()) * 100):.2f}%) | {grouped_df['words'].sum()} reduced to -> {fix_grouped_df['words'].sum()} ")
print(f"Eliminated Sentences = {sentdiff} out of {grouped_df['sentences'].sum()} ({((sentdiff / grouped_df['sentences'].sum()) * 100):.2f}%) | {grouped_df['sentences'].sum()} reduced to -> {fix_grouped_df['sentences'].sum()} ")

Eliminated Words = 71115 out of 716957 (9.92%) | 716957 reduced to -> 645842 
Eliminated Sentences = 5420 out of 52734 (10.28%) | 52734 reduced to -> 47314 


In [52]:
fix_grouped_df

,speaker,text,words,sentences
0,ANDREW,"Flatter for archaeologists. Yeah, it's from t...",27446,3316
1,ANDREWHUBERMAN,"Listen, when it comes to romantic relationshi...",23377,1405
2,BENSHAPIRO,The great light we tell ourselves is that peo...,28319,1621
3,GEORGEHOTZ,Sure. So I think the most obvious way to me i...,23859,2055
4,GUIDOVANROSSUM,How did you pull that off? And what's C Pytho...,23889,1213
5,HANCOCK,We're almost at the edge of history when we g...,26843,2027
6,HUBERMAN,"I need this answer. Big time, foot fetish guy...",26572,2200
7,JOEROGAN,"Joe. What's going on man? Yeah, you're really...",90216,7006
8,JOEY,My brother. How you feeling? Like I'm a new f...,21456,2064
9,KANYE,"Joe Rogan, why can't I check it out? Pichot R...",28416,1634


In [69]:
# Export this raw df
fix_df.to_csv(os.path.join("datasets", "final_df_raw_interventions.csv"), header=True, index=False)

In [79]:
# Export it just with raw sentences

# Preprocess
raw_sent_df = fix_grouped_df.copy(deep=True)
raw_sent_df['sentences'] = raw_sent_df['text'].apply(sent_tokenize)

raw_sent_df = raw_sent_df.explode('sentences').drop(['text', 'words'], axis=1).reset_index(drop=True)

# Keep only sentences with more than three words, others are irrelevant
raw_sent_df['word_count'] = raw_sent_df['sentences'].apply(lambda x: len(x.split()))
raw_sent_df = raw_sent_df[raw_sent_df['word_count'] > 3].drop('word_count', axis=1)

# Export it
raw_sent_df.to_csv(os.path.join("datasets", "final_df_raw_sentences.csv"), header=True, index=False)

In [53]:
# Filter our df taking only speakers that have said more than 20k words
speakers_of_interest = fix_grouped_df[fix_grouped_df['words'] > 20_000]['speaker']
df = fix_df[fix_df['speaker'].isin(speakers_of_interest)][['speaker', 'text']]

In [54]:
df['sentences'] = df['text'].apply(sent_tokenize)
df = df.explode('sentences').drop('text', axis=1).reset_index(drop=True)

In [55]:
# Keep only sentences with more than three words, others are irrelevant
df['word_count'] = df['sentences'].apply(lambda x: len(x.split()))
df = df[df['word_count'] > 3].drop('word_count', axis=1)

In [57]:
def undersample_and_split_dataframe(df, class_column, split_prob=[0.7, 0.15, 0.15]):

    #split parameters
    choices = ['train', 'test', 'val']

    # Group by class and find the smallest class size
    group = df.groupby(class_column)
    smallest_class_size = group.size().min()

    # Sample from each class (we can use with multi-label)
    undersampled_df = pd.DataFrame()
    for _, group_df in group:
        sampled_df = group_df.sample(n=smallest_class_size, replace=False, random_state=1)

        # Perform the split class by class, for the train, test and validation rows to be balanced.
        sampled_df['split'] = np.random.choice(choices, size=len(sampled_df), p=split_prob)
        undersampled_df = pd.concat([undersampled_df, sampled_df], axis=0)

    # Check the distribution
    print(undersampled_df['split'].value_counts(normalize=True))

    return undersampled_df

In [58]:
prepared_df = undersample_and_split_dataframe(df, 'speaker')
prepared_df.columns = ['category', 'title', 'split']
prepared_df = prepared_df.reset_index(drop=True)

split
train    0.706885
test     0.147393
val      0.145722
Name: proportion, dtype: float64


In [59]:
prepared_df.groupby('category').size()

category
ANDREW            352
ANDREWHUBERMAN    352
BENSHAPIRO        352
GEORGEHOTZ        352
GUIDOVANROSSUM    352
HANCOCK           352
HUBERMAN          352
JOEROGAN          352
JOEY              352
KANYE             352
KANYEWEST         352
LEXFRIDMAN        352
MARKZUCKERBERG    352
MILEYCYRUS        352
MRBEAST           352
NILDEGRASSE       352
SNOWDEN           352
dtype: int64

In [60]:
prepared_df.to_csv(os.path.join("datasets", df_name + "_preparado.csv"), header=True, index=False)